# LSTM-FCN SageMaker Algorithm Usage Examples

## Set Up

Load all necessary libraries.

In [1]:
import io
import boto3
import sagemaker
import pandas as pd
import numpy as np

Set up S3.

In [2]:
s3_client = boto3.client('s3')

In [3]:
bucket_name = 'lstm-fcn-bucket'

Set up SageMaker.

In [4]:
sm_role = sagemaker.get_execution_role()

In [5]:
sm_runtime = boto3.client('runtime.sagemaker')

In [6]:
sm_client = boto3.client('sagemaker')

Select the training and inference ECR images.

In [7]:
training_image = '661670223746.dkr.ecr.eu-west-1.amazonaws.com/lstm-fcn-training'

In [8]:
inference_image = '661670223746.dkr.ecr.eu-west-1.amazonaws.com/lstm-fcn-inference'

Select the EC2 instance type.

In [9]:
instance_type = 'ml.c4.xlarge'

Define the model name.

In [10]:
model_name = 'sample-lstm-fcn-model'

Define the endpoint name.

In [11]:
endpoint_name = 'sample-lstm-fcn-endpoint'

Define the job names.

In [12]:
training_job_name = 'sample-lstm-fcn-training'

In [13]:
tuning_job_name = 'sample-lstm-fcn-tuning'

In [14]:
inference_job_name = 'sample-lstm-fcn-inference'

## Training

Run a training job.

In [15]:
estimator = sagemaker.estimator.Estimator(
    role=sm_role,
    image_uri=training_image,
    train_instance_count=1,
    train_instance_type=instance_type,
    output_path=f's3://{bucket_name}/training/',
    hyperparameters={
        'units': 8,
        'dropout': 0.8,
        'filters-1': 4,
        'filters-2': 4,
        'filters-3': 4,
        'kernel-size-1': 3,
        'kernel-size-2': 3,
        'kernel-size-3': 3,
        'batch-size': 64,
        'lr': 0.001,
        'epochs': 10,
    },
)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [16]:
estimator.fit(
    inputs={
        'training': f's3://{bucket_name}/sample-data/train.csv',
        'testing': f's3://{bucket_name}/sample-data/valid.csv'
    },
    job_name=training_job_name
)

INFO:sagemaker:Creating training-job with name: sample-lstm-fcn-training


2023-07-20 18:16:09 Starting - Starting the training job...
2023-07-20 18:16:34 Starting - Preparing the instances for training......
2023-07-20 18:17:25 Downloading - Downloading input data...
2023-07-20 18:17:50 Training - Downloading the training image....................................
2023-07-20 18:23:57 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-07-20 18:24:05,681 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-07-20 18:24:05,682 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-07-20 18:24:05,683 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-07-20 18:24:05,693 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-07-20 18:24:05,706 sagemaker_pytorch_container.training 

## Hyperparameter Tuning

Run a tuning job.

In [17]:
estimator = sagemaker.estimator.Estimator(
    role=sm_role,
    image_uri=training_image,
    train_instance_count=1,
    train_instance_type=instance_type,
    output_path=f's3://{bucket_name}/tuning/',
)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [18]:
tuner = sagemaker.tuner.HyperparameterTuner(
    estimator, 
    objective_metric_name='test:acc', 
    metric_definitions=[
        {
          'Name': 'train:loss',
          'Regex': 'train:loss ([0-9\\.]+)'
        },
        {
          'Name': 'train:acc',
          'Regex': 'train:acc ([0-9\\.]+)'
        },
        {
          'Name': 'test:loss',
          'Regex': 'test:loss ([0-9\\.]+)'
        },
        {
          'Name': 'test:acc',
          'Regex': 'test:acc ([0-9\\.]+)'
        },
    ],
    hyperparameter_ranges={
        'units': sagemaker.parameter.IntegerParameter(4, 16),
        'dropout': sagemaker.parameter.ContinuousParameter(0.1, 0.9),
        'filters-1': sagemaker.parameter.IntegerParameter(4, 16),
        'filters-2': sagemaker.parameter.IntegerParameter(4, 16),
        'filters-3': sagemaker.parameter.IntegerParameter(4, 16),
        'kernel-size-1': sagemaker.parameter.IntegerParameter(3, 7),
        'kernel-size-2': sagemaker.parameter.IntegerParameter(3, 7),
        'kernel-size-3': sagemaker.parameter.IntegerParameter(3, 7),
        'lr': sagemaker.parameter.ContinuousParameter(0.0001, 0.001),
        'batch-size': sagemaker.parameter.CategoricalParameter([32, 64, 128]),
        'epochs': sagemaker.parameter.IntegerParameter(5, 10),
    },
    strategy='Bayesian', 
    objective_type='Maximize', 
    max_jobs=5, 
)

In [19]:
tuner.fit(
    inputs={
        'training': f's3://{bucket_name}/sample-data/train.csv',
        'testing': f's3://{bucket_name}/sample-data/valid.csv'
    },
    job_name=tuning_job_name
)

INFO:sagemaker:Creating hyperparameter tuning job with name: sample-lstm-fcn-tuning


.............................................................................................................................................!


Inspect the results.

In [20]:
tuner.analytics().dataframe()

,batch-size,dropout,epochs,filters-1,filters-2,filters-3,kernel-size-1,kernel-size-2,kernel-size-3,lr,units,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
0,64.0,0.501642,7.0,10.0,6.0,6.0,4.0,5.0,3.0,0.000466,13.0,sample-lstm-fcn-tuning-005-dc09f9bd,Completed,0.8933,2023-07-20 18:36:18+00:00,2023-07-20 18:36:49+00:00,31.0
1,32.0,0.356617,7.0,7.0,4.0,5.0,7.0,4.0,7.0,0.000487,8.0,sample-lstm-fcn-tuning-004-5c55aded,Completed,0.8267,2023-07-20 18:35:35+00:00,2023-07-20 18:36:06+00:00,31.0
2,128.0,0.417961,6.0,15.0,13.0,4.0,3.0,3.0,3.0,0.000129,15.0,sample-lstm-fcn-tuning-003-9be5f289,Completed,0.3133,2023-07-20 18:34:23+00:00,2023-07-20 18:35:28+00:00,65.0
3,128.0,0.430234,7.0,12.0,9.0,9.0,4.0,7.0,4.0,0.000499,7.0,sample-lstm-fcn-tuning-002-90f37842,Completed,0.5733,2023-07-20 18:33:41+00:00,2023-07-20 18:34:12+00:00,31.0
4,32.0,0.597184,5.0,12.0,5.0,15.0,4.0,4.0,3.0,0.000911,5.0,sample-lstm-fcn-tuning-001-83a2d4e0,Completed,0.7800,2023-07-20 18:26:18+00:00,2023-07-20 18:33:17+00:00,419.0


## Batch Transform

Run a batch transform job.

In [21]:
model = sagemaker.model.Model(
    role=sm_role,
    image_uri=inference_image,
    model_data=f's3://{bucket_name}/training/{training_job_name}/output/model.tar.gz',
    name=model_name
)

In [22]:
transformer = model.transformer(
    instance_count=1, 
    instance_type=instance_type,
    accept='text/csv',
    output_path=f's3://{bucket_name}/inference/{inference_job_name}/',
)

INFO:sagemaker:Creating model with name: sample-lstm-fcn-model


In [23]:
transformer.transform(
    data=f's3://{bucket_name}/sample-data/test_data.csv',
    content_type='text/csv',
    job_name=inference_job_name
)

INFO:sagemaker:Creating transform job with name: sample-lstm-fcn-inference


2023-07-20T18:46:55,952 [INFO ] main org.pytorch.serve.servingsdk.impl.PluginsManager - Initializing plugins manager...
2023-07-20T18:46:56,060 [INFO ] main org.pytorch.serve.metrics.configuration.MetricConfiguration - Successfully loaded metrics configuration from /opt/conda/lib/python3.10/site-packages/ts/configs/metrics.yaml
2023-07-20T18:46:56,223 [INFO ] main org.pytorch.serve.ModelServer - 
Torchserve version: 0.8.0
TS Home: /opt/conda/lib/python3.10/site-packages
Current directory: /
Temp directory: /home/model-server/tmp
Metrics config path: /opt/conda/lib/python3.10/site-packages/ts/configs/metrics.yaml
Number of GPUs: 0
Number of CPUs: 4
Max heap size: 1866 M
Python executable: /opt/conda/bin/python3.10
Config file: /etc/sagemaker-ts.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Metrics address: http://127.0.0.1:8082
Model Store: /.sagemaker/ts/models
Initial Models: model=/opt/ml/model
Log dir: /logs
Metrics dir: /logs
Netty thread

Inspect the results.

In [24]:
body = s3_client.get_object(Bucket=bucket_name, Key=f'inference/{inference_job_name}/test_data.csv.out')['Body']
data = body.read().decode('utf-8')
df = pd.read_csv(io.StringIO(data), header=None, index_col=None)
transformer_predictions = df.values.astype(int).flatten()
print(transformer_predictions)

[1 0 2 2 2 1 0 0 2 2 0 2 1 1 1 2 2 0 1 2 0 2 2 2 1 0 0 1 0 2 1 0 2 1 2 2 2
 2 1 1 2 2 2 2 1 0 0 0 2 0 1 1 0 1 2 1 0 1 0 2 2 0 1 1 0 0 1 2 1 2 0 1 2 1
 2 0 1 2 1 2 1 0 0 0 1 1 1 0 2 2 0 0 1 1 1 2 2 0 1 2 0 2 0 2 2 0 2 1 1 2 2
 0 1 1 2 2 0 0 0 2 1 1 0 1 2 0 2 1 0 0 0 2 0 1 0 2 2 0 0 2 0 1 0 1 2 2 0 1
 0 1]


## Real-Time Inference

Deploy an endpoint.

In [25]:
model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    serverless_inference_config=None,
    endpoint_name=endpoint_name
)

INFO:sagemaker:Creating model with name: sample-lstm-fcn-model
INFO:sagemaker:Creating endpoint-config with name sample-lstm-fcn-endpoint
INFO:sagemaker:Creating endpoint with name sample-lstm-fcn-endpoint


-------------!

Invoke the endpoint.

In [26]:
body = s3_client.get_object(Bucket=bucket_name, Key=f'sample-data/test_data.csv')['Body']
data = body.read().decode('utf-8')

In [27]:
response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='text/csv',
    Body=data
)

Inspect the results.

In [28]:
endpoint_predictions = np.array([int(x) for x in response['Body'].read().decode('utf-8').split('\n')[:-1]], dtype=int)
print(endpoint_predictions)

[1 0 2 2 2 1 0 0 2 2 0 2 1 1 1 2 2 0 1 2 0 2 2 2 1 0 0 1 0 2 1 0 2 1 2 2 2
 2 1 1 2 2 2 2 1 0 0 0 2 0 1 1 0 1 2 1 0 1 0 2 2 0 1 1 0 0 1 2 1 2 0 1 2 1
 2 0 1 2 1 2 1 0 0 0 1 1 1 0 2 2 0 0 1 1 1 2 2 0 1 2 0 2 0 2 2 0 2 1 1 2 2
 0 1 1 2 2 0 0 0 2 1 1 0 1 2 0 2 1 0 0 0 2 0 1 0 2 2 0 0 2 0 1 0 1 2 2 0 1
 0 1]


In [29]:
np.isclose(transformer_predictions, endpoint_predictions)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

Delete the endpoint.

In [30]:
sm_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '205aabab-d4e3-4a1a-b0de-50a5a32d1659',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '205aabab-d4e3-4a1a-b0de-50a5a32d1659',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 20 Jul 2023 18:54:29 GMT'},
  'RetryAttempts': 0}}